<a href="https://colab.research.google.com/github/poddubnyoleg/nondualbot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras.backend as K
import sys

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('/content/drive/My Drive/nondualbot/i_am_that.txt', 'r') 
data = file.read() 

In [4]:
data[:100]

'That which permeates all, which nothing transcends and which, like the universal space around us, fi'

In [0]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:

n_chars = len(data)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  988168
Total Vocab:  55


For each chunk of data [seq_length] we take one-hot-encoded dataX array and dataY array (same but shifted +1)

In [0]:
seq_length = 100
overlapping_length = 30
dataX = []
dataY = []
cur_pos = 0
while cur_pos<n_chars-seq_length-1:
  seq_input = np_utils.to_categorical([char_to_int[char] for char in data[cur_pos:cur_pos + seq_length]], num_classes=n_vocab)
  seq_output = np_utils.to_categorical(char_to_int[data[cur_pos + seq_length]],num_classes=n_vocab)
  dataX.append(seq_input)
  dataY.append(seq_output)

  cur_pos+=5

In [8]:
X = np.reshape(dataX, (len(dataX), seq_length, n_vocab))
print(X.shape)

(197614, 100, 55)


In [9]:
y = np.reshape(dataY, (len(dataY), n_vocab))
print(y.shape)

(197614, 55)


In [10]:
X.shape[1], X.shape[2], y.shape[1]

(100, 55, 55)

In [11]:
K.clear_session()

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          319488    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 55)                14135     
Total params: 858,935
Trainable params: 858,935
Non-trainable params: 0
_________________________________________________________________


In [0]:
filepath="/content/drive/My Drive/nondualbot/best_current_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
197614/197614 [==============================] - 622s 3ms/step - loss: 0.6280

Epoch 00001: loss improved from 0.62872 to 0.62796, saving model to /content/drive/My Drive/nondualbot/best_current_weights.hdf5
Epoch 2/100
197614/197614 [==============================] - 621s 3ms/step - loss: 0.6217

Epoch 00002: loss improved from 0.62796 to 0.62174, saving model to /content/drive/My Drive/nondualbot/best_current_weights.hdf5
Epoch 3/100
197614/197614 [==============================] - 622s 3ms/step - loss: 0.6208

Epoch 00003: loss improved from 0.62174 to 0.62081, saving model to /content/drive/My Drive/nondualbot/best_current_weights.hdf5
Epoch 4/100
197614/197614 [==============================] - 622s 3ms/step - loss: 0.6147

Epoch 00004: loss improved from 0.62081 to 0.61467, saving model to /content/drive/My Drive/nondualbot/best_current_weights.hdf5
Epoch 5/100
197614/197614 [==============================] - 622s 3ms/step - loss: 0.6139

Epoch 00005: loss improved fr

In [0]:
# load the network weights
filename = "/content/drive/My Drive/nondualbot/best_current_weights.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
model.evaluate(X, y)

98807/98807 [==============================] - 779s 8ms/step


0.04695190974103574

In [36]:
(np.array([[1,2,3]]*3)).shape

(3, 3)

In [48]:
prediction

array([[9.9999523e-01, 2.0463224e-06, 9.3156482e-16, 2.3285486e-06,
        3.1841680e-07, 1.1315500e-36, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        4.9429824e-36, 1.0919554e-22, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.3220731e-36,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        9.8703578e-27, 0.0000000e+00, 0.0000000e+00, 3.1538452e-32,
        2.3943110e-30, 7.3807023e-17, 2.3995033e-16, 5.8587215e-16,
        3.7597976e-14, 2.5189800e-11, 7.6478038e-16, 5.2172624e-11,
        9.8157481e-15, 2.1217654e-10, 3.3302342e-25, 1.8736244e-16,
        2.3516426e-13, 2.5805555e-15, 7.1146422e-12, 3.6919934e-15,
        9.9358932e-21, 7.0374929e-28, 3.0649214e-13, 2.7722578e-11,
        1.3439056e-13, 7.3349047e-15, 3.3891350e-18, 9.7494195e-18,
        1.6992857e-22, 7.3972104e-12, 1.7295703e-31]], dtype=float32)

In [1]:

# pick a random seed
#start = np.random.randint(0, len(dataX)-1)
#pattern = dataX[start]
#print(''.join([int_to_char[value] for value in pattern_flat]))
question = 'Who am I?'
pattern = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
pattern = np.append(np.zeros(shape=(max(seq_length-pattern.shape[0],0), n_vocab)), pattern, axis=0)

print(question)

# generate characters
for i in range(150):

  prediction = model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)
  
  index = np.argmax(prediction)#np.random.choice(len(prediction[0]), p=prediction[0])#
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern = np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)
  pattern = pattern[1:len(pattern)]
  #print("\nDone.")

NameError: ignored

In [25]:
def sample_prediction(prediction):
  """Get rand index from preds based on its prob distribution.

  Params
  ——
  prediction (array (array)): array of length 1 containing array of probs that sums to 1

  Returns
  ——-
  rnd_idx (int): random index from prediction[0]

  Notes
  —–
  Helps to solve problem of repeated outputs.

  len(prediction) = 1
  len(prediction[0]) >> 1
  """
  X = prediction[0] # sum(X) is approx 1
  rnd_idx = np.random.choice(len(X), p=X)
  return rnd_idx

for i in range(num_outputs):
x = np.reshape(pattern, (1, len(pattern), 1))
x = x / float(n_vocab)
prediction = model.predict(x, verbose=0)
#index = numpy.argmax(prediction)
# per Gustavo’s suggestion, we should not use argmax here
index = sample_prediction(prediction)
result = int_to_char[index]
#seq_in = [int_to_char[value] for value in pattern]
# not sure why seq_in was here
sys.stdout.write(result)
pattern.append(index)
pattern = pattern[1:len(pattern)]
print “\nDone.”


ded. And yet in India every holy man has his tiger, lion, leopard or antelope skin to sit on. Maybe 
e e

which makes the changeful possible. But he cannot give you the conviction. It must come with your ow
   

 all suffering self-created? Yes, as long as there is a separate self to create it. In the end you k
o o

aper with All the horrors going on. Since the world is yourself, how can you explain such misbehavio
r r

scious of not having a body. I see you smoking Exactly so. You see me smoking. Find out for yourself
h h

r butter. Do it correctly and assiduously and the result is sure to come. How can the absolute be th
   

 me how it is done. In selfawareness I see the body and the mind moved by causes beyond my control. 
e e

es. But a general memory of well-being is there. There is a difference in feeling when we say I was 
e e

from the experience to the experiencer and realise the full import of the only true statement you ca
   

ce. Be devoted to your goal -- and devotion t

In [0]:
pattern = dataX[start]
#pattern = np.append(pattern, np_utils.to_categorical(index,num_classes=n_vocab))

In [58]:
np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
patte

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [23]:
np.argmax(dataY[0])

40

In [24]:
dataY[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)